In [ ]:
url = "http://192.168.3.6:8080"
graph_name = "network"
user_id = "65cf8f2ca2605827790739034ae9e58f7efb3968"

In [ ]:
from langgraph_sdk import get_client, get_sync_client

client = get_client(url=url)
sync_client = get_sync_client(url=url)

- 创建Thread （一个对话，表示用户的对话）

In [ ]:
thread = sync_client.threads.create(metadata={"user_id": user_id})

- 发送消息

In [ ]:
config = {"configurable": {"thread_id": thread["thread_id"]}}

input = {
    "messages": {
        "type": "human",
        "content": "把这个地址上的bnb和usdt加入到我的仓位中。0x25Dd4FD54a3c0C70cD780000D9a0eE90d93B8E85",
    },
    "wallet_is_connected": True,
    "chain_id": 56,
    "wallet_address": "0x25Dd4FD54a3c0C70cD780000D9a0eE90d93B8E85",
    "time_zone": "Asia/Shanghai",
    "user_id": user_id,
}
chunks = client.runs.stream(
    thread_id=thread["thread_id"], assistant_id="network", input=input
)

- 打印所有chunk，包含所有处理过程事件（数据，例如，工具返回值）

In [ ]:
async for chunk in chunks:
    print(f"Event:{chunk.event}")
    print(chunk.data)
    print("=" * 100)

- 查询用户所有的对话

In [ ]:
threads = await client.threads.search(metadata={"user_id": user_id}, limit=100)
for thread in threads:
    print(thread.values())

dict_values(['8bbf1119-6ef7-4a6c-af85-37704e89ba92', '2025-06-30T09:25:50.928874+00:00', '2025-06-30T09:27:44.930474+00:00', {'user_id': 'aaa', 'graph_id': 'network', 'assistant_id': '8ccc85cf-dd91-51bd-8296-6f2ff7ceeced'}, 'idle', {'configurable': {'user-agent': 'langgraph-sdk-py/0.1.70', 'x-request-id': '180194b8-319c-4523-9b93-5638597e3410', 'langgraph_auth_user': None, 'langgraph_auth_user_id': '', 'langgraph_auth_permissions': [], 'langgraph_request_id': '180194b8-319c-4523-9b93-5638597e3410', '__request_start_time_ms__': 1751275580153, '__after_seconds__': 0}}, {'messages': [{'content': '把这个地址上的bnb和usdt加入到我的仓位中。0x25Dd4FD54a3c0C70cD780000D9a0eE90d93B8E85', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'd5c237ee-8562-4787-9687-23323377d6e9', 'example': False}, {'content': [{'text': '我来帮您将这个地址上的BNB和USDT添加到您的投资组合中。首先让我检查一下您现有的资产来源。', 'type': 'text', 'index': 0}, {'id': 'toolu_01GCGZstPPBBLCTpGbCx9R1E', 'input': {}, 'name': 'get_user_asset_sour